In [ ]:
import os
import tensorflow as tf
import tensorflow_recommenders as tfrs
import tensorflow_model_analysis as tfma

from tfx import components
from tfx.proto import example_gen_pb2, trainer_pb2,pusher_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

In [ ]:
PIPELINE_ROOT = "pipeline"
SCHEMA_PIPELINE_NAME = "movies-rec"
SERVING_MODEL_DIR = "serving_model_dir"

In [ ]:
DATA_ROOT = "data/movie"

In [ ]:
context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

In [ ]:
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=9),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=1),
    ])
)

example_gen = components.CsvExampleGen(
    input_base=DATA_ROOT,
    output_config=output
)
context.run(example_gen)

In [ ]:
statistics_gen = components.StatisticsGen(
    examples=example_gen.outputs["examples"]
)
context.run(statistics_gen)

In [ ]:
context.show(statistics_gen.outputs["statistics"])

In [ ]:
schema_gen = components.SchemaGen(
    statistics=statistics_gen.outputs["statistics"]
)
context.run(schema_gen)

In [ ]:
example_validator = components.ExampleValidator(
    statistics=statistics_gen.outputs["statistics"],
    schema=schema_gen.outputs["schema"],
)
context.run(example_validator)

In [ ]:
context.show(example_validator.outputs["anomalies"])

In [ ]:
TRANSFORM_MODULE_FILE = "movie_transform.py"

In [ ]:
transform = components.Transform(
    examples=example_gen.outputs["examples"],
    schema=schema_gen.outputs["schema"],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
context.run(transform)

In [ ]:
context.show(transform.outputs["post_transform_schema"])

In [ ]:
TRAINER_MODULE_FILE = "movie_trainer.py"